# Training a GCN with NAGL

## Imports

In [1]:
import os
import pickle
from pathlib import Path

import tqdm

from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint, TQDMProgressBar

from openff.toolkit import Molecule


from openff.nagl import GNNModel
from openff.nagl.features import atoms, bonds
from openff.nagl.nn.dataset import DGLMoleculeLightningDataModule
from openff.nagl.storage.record import MoleculeRecord
from openff.nagl.storage import MoleculeStore


/home/joshmitchell/Documents/openff/nagl/.soap/examples/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Configuration

In [2]:
output_directory = Path('output') # The path to an output directory

n_epochs = 200 # Number of epochs"

n_gpus = 1 # Number of gpus

partial_charge_method = "am1bcc" # Method

# dataset_paths = [Path("alkanes.sqlite")]
dataset_paths = [Path("../prepare-dataset/alkanes_by_josh.sqlite")]


## Put together a test dataset

In [3]:
test_smiles = [
    "CCCCCCC",
    "CC(C)C(C)C",
    "CC(C)(C)C",
]

records = []

for smiles in tqdm.tqdm(test_smiles, desc="Labeling molecules"):
    mol = Molecule.from_smiles(smiles, allow_undefined_stereo=True)
    record = MoleculeRecord.from_openff(
        mol,
        partial_charge_methods=["am1bcc", "am1"],
        generate_conformers=True,
        n_conformer_pool=500,
        n_conformers=10,
        rms_cutoff=0.05,
    )
    records.append(record)
    
test_set_path = Path("my_first_test_dataset.sqlite")
if test_set_path.exists():
    test_set_path.unlink()

store = MoleculeStore(test_set_path)
store.store(records)

storing grouped records: 100%|███████████████████████████████████████| 3/3 [00:00<00:00, 325.29it/s]


## Create the model

In [4]:
atom_features = (
    atoms.AtomicElement(["C", "O", "H", "N", "S", "F", "Br", "Cl", "I", "P"]),
    atoms.AtomConnectivity(),
    atoms.AtomAverageFormalCharge(),
    atoms.AtomHybridization(),
    atoms.AtomInRingOfSize(3),
    atoms.AtomInRingOfSize(4),
    atoms.AtomInRingOfSize(5),
    atoms.AtomInRingOfSize(6),
)

bond_features = (
    bonds.BondInRingOfSize(3),
    bonds.BondInRingOfSize(4),
    bonds.BondInRingOfSize(5),
    bonds.BondInRingOfSize(6),
)

In [5]:

model = GNNModel(
    convolution_architecture="SAGEConv",
    n_convolution_hidden_features=128,
    n_convolution_layers=3,
    n_readout_hidden_features=128,
    n_readout_layers=4,
    activation_function="ReLU",
    postprocess_layer="compute_partial_charges",
    readout_name=f"{partial_charge_method}-charges",
    learning_rate=0.001,
    atom_features=atom_features,
    bond_features=bond_features,
)

## Specify the training, validation and test data

- **training**: Data the model is trained against

- **validation**: Data used to validate the model as it is trained

- **tests**: Data used to test that the final model is good

In [21]:
!rm -r output data

data_module = DGLMoleculeLightningDataModule(
    atom_features=atom_features,
    bond_features=bond_features,
    partial_charge_method=partial_charge_method,
    training_set_paths=dataset_paths,
    validation_set_paths=dataset_paths,
    test_set_paths=[test_set_path],
)

rm: cannot remove 'output': No such file or directory


## Train the model

In [22]:
os.makedirs(str(output_directory), exist_ok=True)

logger = TensorBoardLogger(output_directory)

callbacks = [TQDMProgressBar(), ModelCheckpoint(save_top_k=1, monitor="val_loss")]

trainer = Trainer(
#     gpus=n_gpus,
    min_epochs=n_epochs,
    max_epochs=n_epochs,
    logger=logger,
    callbacks=callbacks,
)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [23]:
trainer.fit(
    model, 
    datamodule=data_module, 
)

featurizing molecules: 100%|██████████████████████████████████████████| 3/3 [00:00<00:00, 52.01it/s]
Missing logger folder: output/lightning_logs

  | Name               | Type              | Params
---------------------------------------------------------
0 | convolution_module | ConvolutionModule | 72.3 K
1 | readout_modules    | ModuleDict        | 66.3 K
---------------------------------------------------------
138 K     Trainable params
0         Non-trainable params
138 K     Total params
0.555     Total estimated model params size (MB)


Epoch 0:  50%|████████████████                | 1/2 [00:00<00:00, 125.44it/s, loss=0.00182, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 1:  50%|████████████████▌                | 1/2 [00:00<00:00, 134.98it/s, loss=0.0267, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 2:  50%|████████████████▌                | 1/2 [00:00<00:00, 132.55it/s, loss=0.0275, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 3:  50%|█████████████████                 | 1/2 [00:00<00:00, 133.44it/s, loss=0.021, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 4:  50%|████████████████▌                | 1/2 [00:00<00:00, 144.05it/s, loss=0.0197, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 5:  50%|████████████████▌                | 1/2 [00:00<00:00, 144.32it/s, loss=0.0192, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 6:  50%|████████████████▌                | 1/2 [00:00<00:00, 140.87it/s, loss=0.0184, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 7:  50%|█████████████████                 | 1/2 [00:00<00:00, 140.55it

Epoch 18:  50%|████████████████                | 1/2 [00:00<00:00, 142.55it/s, loss=0.0103, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 19:  50%|███████████████▌               | 1/2 [00:00<00:00, 137.99it/s, loss=0.00983, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 20:  50%|███████████████▌               | 1/2 [00:00<00:00, 146.52it/s, loss=0.00986, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 21:  50%|███████████████▌               | 1/2 [00:00<00:00, 146.56it/s, loss=0.00735, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 22:  50%|███████████████▌               | 1/2 [00:00<00:00, 145.69it/s, loss=0.00594, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 23:  50%|███████████████▌               | 1/2 [00:00<00:00, 143.15it/s, loss=0.00596, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 24:  50%|███████████████▌               | 1/2 [00:00<00:00, 141.47it/s, loss=0.00529, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 25:  50%|███████████████▌               | 1/2 [00:00<00:00, 142.71it/s

Epoch 36:  50%|███████████████▌               | 1/2 [00:00<00:00, 112.49it/s, loss=0.00151, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 37:  50%|███████████████▌               | 1/2 [00:00<00:00, 137.46it/s, loss=0.00135, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 38:  50%|████████████████                | 1/2 [00:00<00:00, 136.31it/s, loss=0.0012, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 39:  50%|███████████████▌               | 1/2 [00:00<00:00, 139.73it/s, loss=0.00121, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 40:  50%|███████████████▌               | 1/2 [00:00<00:00, 136.71it/s, loss=0.00115, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 41:  50%|███████████████▌               | 1/2 [00:00<00:00, 140.73it/s, loss=0.00112, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 42:  50%|███████████████▌               | 1/2 [00:00<00:00, 114.07it/s, loss=0.00113, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 43:  50%|███████████████▌               | 1/2 [00:00<00:00, 144.01it/s

Epoch 54:  50%|███████████████▌               | 1/2 [00:00<00:00, 120.13it/s, loss=0.00103, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 55:  50%|███████████████▌               | 1/2 [00:00<00:00, 112.78it/s, loss=0.00101, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 56:  50%|███████████████               | 1/2 [00:00<00:00, 111.75it/s, loss=0.000973, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 57:  50%|███████████████               | 1/2 [00:00<00:00, 120.57it/s, loss=0.000958, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 58:  50%|████████████████▌                | 1/2 [00:00<00:00, 118.50it/s, loss=0.001, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 59:  50%|███████████████               | 1/2 [00:00<00:00, 118.00it/s, loss=0.000979, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 60:  50%|████████████████                | 1/2 [00:00<00:00, 140.02it/s, loss=0.0011, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 61:  50%|███████████████▌               | 1/2 [00:00<00:00, 145.40it/s

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Epoch 180:  50%|███████████████               | 1/2 [00:00<00:00, 147.08it/s, loss=0.00165, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 181:  50%|███████████████               | 1/2 [00:00<00:00, 131.92it/s, loss=0.00156, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 182:  50%|███████████████               | 1/2 [00:00<00:00, 141.76it/s, loss=0.00158, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 183:  50%|███████████████               | 1/2 [00:00<00:00, 143.66it/s, loss=0.00163, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 184:  50%|███████████████               | 1/2 [00:00<00:00, 145.82it/s, loss=0.00166, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 185:  50%|███████████████               | 1/2 [00:00<00:00, 142.01it/s, loss=0.00168, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 186:  50%|███████████████               | 1/2 [00:00<00:00, 147.11it/s, loss=0.00163, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 187:  50%|███████████████               | 1/2 [00:00<00:00, 147.72it/s

Epoch 198:  50%|███████████████               | 1/2 [00:00<00:00, 146.03it/s, loss=0.00163, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 199:  50%|███████████████               | 1/2 [00:00<00:00, 147.50it/s, loss=0.00155, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 199: 100%|██████████████████████████████| 2/2 [00:00<00:00, 119.03it/s, loss=0.00155, v_num=0]

`Trainer.fit` stopped: `max_epochs=200` reached.


Epoch 199: 100%|██████████████████████████████| 2/2 [00:00<00:00, 113.16it/s, loss=0.00155, v_num=0]


In [24]:
trainer.test(model, data_module)

Testing DataLoader 0: 100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 215.41it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │   0.0028741960413753986   │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.0028741960413753986}]

## Results!

In [10]:
print("--- Best model ---")
print(callbacks[1].best_model_path)
print(callbacks[1].best_model_score)
metrics_file = Path(output_directory) / "metrics.pkl"
with open(str(metrics_file), "wb") as f:
    metrics = (trainer.callback_metrics, trainer.logged_metrics)
    pickle.dump(metrics, f)

print(f"Wrote metrics to {str(metrics_file)}")

--- Best model ---
output/lightning_logs/version_0/checkpoints/epoch=155-step=156.ckpt
tensor(0.0005)
Wrote metrics to output/metrics.pkl


In [11]:
propane = Molecule.from_smiles("CCC")

model.compute_property(propane)

tensor([[-0.0893],
        [-0.0795],
        [-0.0893],
        [ 0.0311],
        [ 0.0311],
        [ 0.0311],
        [ 0.0357],
        [ 0.0357],
        [ 0.0311],
        [ 0.0311],
        [ 0.0311]], grad_fn=<CatBackward0>)

In [12]:
propane.assign_partial_charges(partial_charge_method)
propane.partial_charges

Magnitude,[-0.09227999977090141 -0.08139999888160011 -0.09227999977090141 0.03215999969027259 0.03215999969027259 0.03215999969027259 0.036500000140883705 0.036500000140883705 0.03215999969027259 0.03215999969027259 0.03215999969027259]
Units,elementary_charge


In [14]:
mol = Molecule.from_smiles(test_smiles[0])

model.compute_property(mol)

tensor([[-0.0886],
        [-0.0774],
        [-0.0757],
        [-0.0756],
        [-0.0757],
        [-0.0774],
        [-0.0886],
        [ 0.0317],
        [ 0.0317],
        [ 0.0317],
        [ 0.0367],
        [ 0.0367],
        [ 0.0371],
        [ 0.0371],
        [ 0.0371],
        [ 0.0371],
        [ 0.0371],
        [ 0.0371],
        [ 0.0367],
        [ 0.0367],
        [ 0.0317],
        [ 0.0317],
        [ 0.0317]], grad_fn=<CatBackward0>)

In [16]:
mol.assign_partial_charges(partial_charge_method)
mol.partial_charges

Magnitude,[-0.09234000029771225 -0.07955999704806702 -0.07857999983041183 -0.07912000301091568 -0.07857999983041183 -0.07955999704806702 -0.09234000029771225 0.03220000085623368 0.03220000085623368 0.03220000085623368 0.03793999892861947 0.03793999892861947 0.039289999429298485 0.039289999429298485 0.03897999939711198 0.03897999939711198 0.039289999429298485 0.039289999429298485 0.03793999892861947 0.03793999892861947 0.03220000085623368 0.03220000085623368 0.03220000085623368]
Units,elementary_charge


In [19]:
list(model.compute_property(mol)), list(mol.partial_charges)

([tensor([-0.0886], grad_fn=<UnbindBackward0>),
  tensor([-0.0774], grad_fn=<UnbindBackward0>),
  tensor([-0.0757], grad_fn=<UnbindBackward0>),
  tensor([-0.0756], grad_fn=<UnbindBackward0>),
  tensor([-0.0757], grad_fn=<UnbindBackward0>),
  tensor([-0.0774], grad_fn=<UnbindBackward0>),
  tensor([-0.0886], grad_fn=<UnbindBackward0>),
  tensor([0.0317], grad_fn=<UnbindBackward0>),
  tensor([0.0317], grad_fn=<UnbindBackward0>),
  tensor([0.0317], grad_fn=<UnbindBackward0>),
  tensor([0.0367], grad_fn=<UnbindBackward0>),
  tensor([0.0367], grad_fn=<UnbindBackward0>),
  tensor([0.0371], grad_fn=<UnbindBackward0>),
  tensor([0.0371], grad_fn=<UnbindBackward0>),
  tensor([0.0371], grad_fn=<UnbindBackward0>),
  tensor([0.0371], grad_fn=<UnbindBackward0>),
  tensor([0.0371], grad_fn=<UnbindBackward0>),
  tensor([0.0371], grad_fn=<UnbindBackward0>),
  tensor([0.0367], grad_fn=<UnbindBackward0>),
  tensor([0.0367], grad_fn=<UnbindBackward0>),
  tensor([0.0317], grad_fn=<UnbindBackward0>),
  tens